In [66]:
include("LiPoSID.jl")
using LinearAlgebra

We can vectorize Kraus operator sum representation:

$\mathcal{E}(\rho) = \sum_k E_k \rho E_k^\dagger$

$\operatorname{vec}(E \rho E^\dagger) = E^* \otimes E \operatorname{vec}(\rho)$

$\operatorname{vec} \mathcal{E}(\rho) = \sum_k  E_k^* \otimes E_k \operatorname{vec}(\rho) $

$\frac{d}{dt}\operatorname{vec}(\rho) = \frac{\log (\sum_k  E_k^* \otimes E_k )}{\Delta t} \operatorname{vec}(\rho)$

$\mathcal{L}^{Kraus} = \frac{\log (\sum_k  E_k^* \otimes E_k )}{\Delta t}$

In [78]:
function ⊗(A, B)
    return kron(A,B)
end

function L_from_Kraus(E, Δt)
   log(sum(conj(e) ⊗ e for e in E))/Δt
end

function A_from_Kraus(E)
    sum(conj(e) ⊗ e for e in E)
end

function A_from_B4D(Aᵇ⁴)

    M = [ 0  0   1  1
          1  im  0  0 
          1  -im  0  0
          0  0  -1  1 ]/2
   
    A =   M * Aᵇ⁴ * inv(M)
   
    return Aᵇ⁴
   
   end

A_from_B4D (generic function with 1 method)

In [5]:
using DynamicPolynomials

Anmplitude damping

$\begin{bmatrix}
1 & 0 \\ 0 & \sqrt{1-\gamma}
\end{bmatrix}, \quad \begin{bmatrix}
0 & \sqrt{\gamma} \\ 0 & 0 \end{bmatrix}$

In [40]:
using Symbolics
@variables γ

Eˢʸᵐᵇ = []
push!(Eˢʸᵐᵇ, [1 0
              0 sqrt(1-γ)])
        
push!(Eˢʸᵐᵇ, [0 sqrt(γ)
              0  0 ])

A_from_Kraus(Eˢʸᵐᵇ)

4×4 Matrix{Num}:
 1            0            0      sqrt(γ)^2
 0  sqrt(1 - γ)            0              0
 0            0  sqrt(1 - γ)              0
 0            0            0  sqrt(1 - γ)^2

In [62]:
γ = 0.123456789
E = []
push!(E, [1 0
          0 sqrt(1-γ)])
        
push!(E, [0 sqrt(γ)
          0  0 ])

sum(e'e for e in E)

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

In [48]:
ρᵍ₀ = [1. 0]' * [1 0]
ρᵉ₀ = [0. 1]' * [0 1]
ρˣ₀ = [1 1]' * [1 1]/2
ρʸ₀ = [1 -im]' * [1 -im]/2

2×2 Matrix{ComplexF64}:
 0.5+0.0im  0.0-0.5im
 0.0+0.5im  0.5+0.0im

In [81]:
t_steps = 1000
ρᵍ = LiPoSID.timeevolution_kraus(t_steps, ρᵍ₀, E)
ρᵉ = LiPoSID.timeevolution_kraus(t_steps, ρᵉ₀, E)
ρˣ = LiPoSID.timeevolution_kraus(t_steps, ρˣ₀, E)
ρʸ = LiPoSID.timeevolution_kraus(t_steps, ρʸ₀, E)

A = LiPoSID.direct_DMD_01XY_b4_A([ρᵉ, ρᵍ, ρˣ, ρʸ])
A[abs.(A) .< 1e-5] .= 0
A

4×4 Matrix{Float64}:
 0.936239  0.0  0.0       0.0
 0.0       0.0  0.0       0.0
 0.0       0.0  0.876543  0.123457
 0.0       0.0  0.0       1.0

In [80]:
A_from_B4D(A)

4×4 Matrix{Float64}:
 0.936239  0.0  0.0       0.0
 0.0       0.0  0.0       0.0
 0.0       0.0  0.876543  0.123457
 0.0       0.0  0.0       1.0

In [70]:
function bloch4(ρ)

    b = convert.(Float64, [ ρ[1,2] + ρ[2,1],
                           (ρ[1,2] - ρ[2,1])*im,
                            ρ[1,1] - ρ[2,2],
                            ρ[1,1] + ρ[2,2]       ])

end

function dm_b4(b) 

    ρ = [ 1+b[3]         b[2]-im*b[3]
          b[2]+im*b[3]   1-b[3]       ]/2

end

function 𝓔(ρ₀, A)
    ρᶠ = dm_b4(A * bloch4(ρ₀))
    return ρᶠ
end

function QPT(A)

    ρ₁ = 𝓔(ρᵍ₀, A)
    ρ₄ = 𝓔(ρᵉ₀, A)
    ρ₂ = 𝓔(ρˣ₀, A) + im*𝓔(ρʸ₀, A) - (1-im)*(ρ₁+ρ₄)/2 # with "-" sign χ can be diagonalized ???
    ρ₃ = 𝓔(ρˣ₀, A) - im*𝓔(ρʸ₀, A) - (1+im)*(ρ₁+ρ₄)/2 

    #ρ₂ = 𝓔(ρˣ₀, Ac) - im*𝓔(ρʸ₀, Ac) - (1-im)*(ρ₁+ρ₄)/2 # with "-" sign χ can be diagonalized ???
    #ρ₃ = 𝓔(ρˣ₀, Ac) + im*𝓔(ρʸ₀, Ac) - (1+im)*(ρ₁+ρ₄)/2 

    σₓ = [ 0  1
           1  0 ]  # X gate

    Λ = [ I  σₓ
          σₓ  -I ] / 2

    #Ρ = [ρ₁ ρ₃ 
    #     ρ₂ ρ₄] 

    Ρ = [ρ₁ ρ₂
         ρ₃ ρ₄] 
     
    χ = Λ*Ρ*Λ

    return χ 
end

χ = QPT(A)
d, U = eigen(χ)

U * diagm(d) * U' ≈ χ

true

In [97]:
function operator_sum(χ)

    E₀ = [1.  0
      0   1 ]
    E₁ = [ 0.  1
        1  0 ]  #  σₓ  or X gate
    E₂ = -im * [ 0  -im
                im  0  ]
    E₃ = [ 1  0
        0 -1 ]
    Ẽ = [E₀, E₁, E₂, E₃] 

    d, U = eigen(χ)
    #@assert U * diagm(d) * U' ≈ χ
    E = []
    for i in 1:size(U)[2]
        Eᵢ = sqrt(d[i] * sum(U[i,j] * Ẽ[j] for j in 1:size(U)[1]))
        Eᵢ[abs.(Eᵢ) .< 1e-5] .= 0
        push!(E, Eᵢ)
    end
    return E
end

Eᵗᵒᵐ = operator_sum(χ)

sum(e'e for e in Eᵗᵒᵐ)

2×2 Matrix{ComplexF64}:
    2.4906+0.0im       -0.108717+0.201893im
 -0.108717-0.201893im    1.92872+0.0im

In [90]:
Eᵗᵒᵐ[1]

2×2 Matrix{ComplexF64}:
 -9.39881e-8-0.893423im       1.48162-2.26907e-7im
    0.370068-5.66752e-8im  1.36826e-7+0.613707im

In [91]:
E[1]

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  0.936239

In [92]:
Eᵗᵒᵐ[2]

2×2 Matrix{ComplexF64}:
   0.260715-6.15515e-9im  -2.54953e-9+0.130262im
 1.23192e-8-0.62942im         0.31448-5.10283e-9im

In [94]:
E[2]

2×2 Matrix{Float64}:
 0.0  0.351364
 0.0  0.0

In [95]:
Eᵗᵒᵐ[3]

2×2 Matrix{ComplexF64}:
 2.90463e-9+0.213552im        0.427416-1.20313e-9im
   0.515559-1.45126e-9im  -6.01128e-10-1.03187im